<a href="https://colab.research.google.com/github/AeroEng16/GNN_learning/blob/main/DeepmindDatasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# To Do



*   Convert Cells to Edges
*   Convert data to torch geometric data instance (and save to .pt file) and convert to MATLAB readable format
*   Normalise the data
*   Implement meshgraphnet
  * Encoder for node and edges
  * Processor - message passing, aggregation and update
  * Decoder - updated nodes to velocities
* Setup train, test and optimisation functions and assess.




# **Loading Deepmind Research Data**

In [1]:
!git clone https://github.com/google-deepmind/deepmind-research
import tensorflow as tf
import plotly.express as px
import pandas as pd
import numpy as np
import scipy.interpolate
import plotly.graph_objects as go
import torch
from scipy.io import savemat


Cloning into 'deepmind-research'...
remote: Enumerating objects: 3046, done.
remote: Counting objects: 100% (201/201), done.
remote: Compressing objects: 100% (168/168), done.
remote: Total 3046 (delta 140), reused 33 (delta 33), pack-reused 2845 (from 4)
Receiving objects: 100% (3046/3046), 92.07 MiB | 38.67 MiB/s, done.
Resolving deltas: 100% (1568/1568), done.


In [2]:
!mkdir -p CylinderData


In [3]:
!bash deepmind-research/meshgraphnets/download_dataset.sh cylinder_flow CylinderData/

--2025-08-07 16:30:58--  https://storage.googleapis.com/dm-meshgraphnets/cylinder_flow/meta.json
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.121.207, 209.85.145.207, 64.233.179.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.121.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 883 [application/octet-stream]
Saving to: ‘CylinderData//cylinder_flow/meta.json’

CylinderData//cylin 100%[===================>]     883  --.-KB/s    in 0s      

2025-08-07 16:30:58 (13.3 MB/s) - ‘CylinderData//cylinder_flow/meta.json’ saved [883/883]

--2025-08-07 16:30:58--  https://storage.googleapis.com/dm-meshgraphnets/cylinder_flow/train.tfrecord
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.121.207, 209.85.145.207, 64.233.179.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.121.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13645805387 (1

## Define function to Decode Tfrecord file

In [16]:
def extract_fn(data_record):
    features = {
        "cells": tf.io.FixedLenFeature([], tf.string),
        "mesh_pos": tf.io.FixedLenFeature([], tf.string),
        "node_type": tf.io.FixedLenFeature([], tf.string),
        "velocity": tf.io.FixedLenFeature([], tf.string),
        "pressure": tf.io.FixedLenFeature([], tf.string),
    }
    sample = tf.io.parse_single_example(data_record, features)
    sample["cells"] = tf.reshape(tf.io.decode_raw(sample["cells"], tf.int32), [-1, 3])
    sample["mesh_pos"] = tf.reshape(tf.io.decode_raw(sample["mesh_pos"], tf.float32), [-1, 2])
    sample["node_type"] = tf.reshape(tf.io.decode_raw(sample["node_type"], tf.int32), [-1, 1])
    sample["velocity"] = tf.reshape(tf.io.decode_raw(sample["velocity"], tf.float32), [600, -1, 2])
    sample["pressure"] = tf.reshape(tf.io.decode_raw(sample["pressure"], tf.float32), [600, -1, 1])
    return sample

filename = "/content/CylinderData/cylinder_flow/train.tfrecord"
dataset = tf.data.TFRecordDataset(filename)
dataset = dataset.map(extract_fn)
cellData= []
meshPosData = []
nodeTypeData = []
velocityData = []
pressureData = []
i = 0
## Code below pulls first example from dataset
#for example in dataset.take(1):
#  test = example
##Code below loops through each sample in the dataset and saves in a matlab readable format, then deletes variable to avoid running out of RAM
# for count,example in enumerate(dataset):
#    cellData.append(torch.tensor(example["cells"].numpy()))
# #torch.save(cellData,'cellData_train.pt')
# cellDataDict = {'cellData': cellData}
# savemat('cellData.mat',cellDataDict)
# del cellData
# del cellDataDict

##Code below loops through each sample in the dataset and saves in a matlab readable format, then deletes variable to avoid running out of RAM
# for count,example in enumerate(dataset):
#   meshPosData.append(torch.tensor(example["mesh_pos"].numpy()))

# meshPosDataDict = {'meshPosData': meshPosData}
# savemat('meshPosData.mat',meshPosDataDict)
# del meshPosData
# del meshPosDataDict

##Code below loops through each sample in the dataset and saves in a matlab readable format, then deletes variable to avoid running out of RAM
# for count,example in enumerate(dataset):
#   nodeTypeData.append(torch.tensor(example["node_type"].numpy()))

# nodeTypeDataDict = {'nodeTypeData': nodeTypeData}
# savemat('nodeTypeData.mat',nodeTypeDataDict)
# del nodeTypeData
# del nodeTypeDataDict

##Code below loops through each sample in the dataset and saves in a matlab readable format, then deletes variable to avoid running out of RAM
for count,example in enumerate(dataset):
  currentCaseDict = {}
  currentCase = example["velocity"].numpy()
  for j in range(0,currentCase.shape[0]):
    currentCaseDict['Frame'+str(j)] = currentCase[j]

  if count > 1:
    break
  velocityData.append(torch.tensor(example["velocity"].numpy()))

velocityDataDict = {'velocityData': velocityData}
savemat('velocityData.mat',velocityDataDict)
# del velocityData
# del velocityDataDict

# ##Code below loops through each sample in the dataset and saves in a matlab readable format, then deletes variable to avoid running out of RAM
# for count,example in enumerate(dataset):
#   pressureData.append(torch.tensor(example["pressure"].numpy()))

# pressureDataDict = {'pressureData': pressureData}
# savemat('pressureData.mat',pressureDataDict)
# del pressureData
# del pressureDataDict


In [17]:
currentCaseDict

{'Frame0': array([[ 4.5456153e-01,  2.7073536e-03],
        [ 4.5160475e-01, -2.8692564e-04],
        [ 4.6459970e-01,  5.7606087e-03],
        ...,
        [ 9.1049217e-02,  1.4545709e-02],
        [ 0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00]], dtype=float32),
 'Frame1': array([[ 4.6129841e-01,  1.8765632e-04],
        [ 4.6208143e-01, -9.2113734e-04],
        [ 4.1371292e-01, -9.7539080e-03],
        ...,
        [ 5.2487027e-02,  7.7340398e-03],
        [ 0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00]], dtype=float32),
 'Frame2': array([[ 0.4592514 , -0.00519329],
        [ 0.45284873, -0.00197687],
        [ 0.36214653, -0.01583783],
        ...,
        [ 0.03996341,  0.00508616],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ]], dtype=float32),
 'Frame3': array([[ 0.44950622, -0.0112669 ],
        [ 0.43616086, -0.00191229],
        [ 0.32459238, -0.01878356],
        ...,
        [ 0.03352614,  0.0035

In [ ]:
listVals = list(velocityDataDict.values())
for i in listVals[0]:
  print(i.shape)


torch.Size([600, 1876, 2])
torch.Size([600, 1876, 2])
torch.Size([600, 1854, 2])


### Convert a single Example to  A Df and Plot a Contour

In [ ]:
meshDf = pd.DataFrame(test["mesh_pos"].numpy(),columns=['X','Y'])
meshDf['Z'] = 0
velocityDf = pd.DataFrame(test["velocity"][300].numpy(),columns=['U','V'])
nodeTypeDf = pd.DataFrame(test['node_type'].numpy(),columns=['NodeType'])
cellsDf = pd.DataFrame(test['cells'].numpy(),columns=['i','j','k'])

allFramesVelocity = pd.DataFrame()
for frame in range(0,600):
  tempDf1 = pd.DataFrame(test["velocity"][frame].numpy(),columns = ['U','V'])
  tempDf1['frame'] = frame
  tempDf2 = pd.DataFrame(test["mesh_pos"].numpy(),columns=['X','Y'])
  tempDf2['Z'] = 0
  tempDf = pd.concat([tempDf2,tempDf1],axis=1)
  allFramesVelocity = pd.concat([allFramesVelocity,tempDf],axis=0)

combinedDf = pd.concat([meshDf,velocityDf,nodeTypeDf],axis=1)

[XGrid,YGrid] = np.meshgrid(combinedDf.X,combinedDf.Y)
ZGrid = scipy.interpolate.griddata((combinedDf.X,combinedDf.Y), combinedDf.U, (XGrid,YGrid), method='linear')
combinedDf.head()


,X,Y,Z,U,V,NodeType
0,0.000000,0.015771,0,0.077919,0.000000,4
1,0.000000,0.007821,0,0.039421,0.000000,4
2,0.015316,0.015751,0,0.077574,0.000073,0
3,0.000000,0.030556,0,0.145302,0.000000,4
4,0.018191,0.027152,0,0.129856,0.000203,0


In [ ]:
# frames = []
# startFrame = 0
# endFrame = 600
# for i in range (startFrame,endFrame):
#   frames.append(go.Frame(
#       data = [go.Mesh3d(
#           x = meshDf.X,
#                     y = meshDf.Y,
#                     z = meshDf.Z,
#                     i = cellsDf.i,
#                     j = cellsDf.j,
#                     k = cellsDf.k,
#                     intensity = allFramesVelocity[allFramesVelocity.frame == i].U,
#                     colorbar = dict(title="U",tickformat="digit"),
#                     colorscale="jet",
#                     opacity = 1,
#                     showscale=True,
#       )]
#   ))

# fig = go.Figure(
#     data = go.Mesh3d(
#                         x = meshDf.X,
#                         y = meshDf.Y,
#                         z = meshDf.Z,
#                         i = cellsDf.i,
#                         j = cellsDf.j,
#                         k = cellsDf.k,
#                         intensity = allFramesVelocity[allFramesVelocity.frame == startFrame].U,
#                         colorbar = dict(title="U",tickformat="digit"),
#                         colorscale="jet",
#                         opacity = 1,
#                         showscale=True,

#                     ),
#     frames = frames,
#     layout=go.Layout(
#     updatemenus=[dict(
#         type="buttons",
#         buttons=[dict(label="Play",
#                       method="animate",
#                       args=[None, {
#                                     "frame": {"duration": 30, "redraw": True},
#                                     "transition": {"duration": 5}
#                                 }]
#                       )])]
#     ),

# )
# camera = dict(
#     eye=dict(x=0, y=0, z=0.1),
#     up=dict(x=0, y=1., z=0),
#     )


# fig.update_layout(scene_camera=camera,
#                   scene=dict(xaxis = dict(visible=False),
#         yaxis = dict(visible=False),
#         zaxis = dict(visible=False)),)

# #fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 30
# #fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 5

# fig.update_scenes(camera_projection_type='orthographic')



# fig.show()
# fig.write_html('test.html')